## INTRODUCTION

This solution demonstrates how to leverage Snowflake's AISQL capabilities to analyze ad campaign images and impression data. Through this notebook, you'll learn how to use AI-powered functions to extract insights, classify ad content, generate taglines, and aggregate customer feedback—all using familiar SQL syntax.

By expressing everything in SQL, this approach uniquely bridges structured and unstructured data analysis. It enables seamless integration of multiple data modalities, allowing marketers to analyze images alongside traditional metrics in a single, unified workflow without switching between specialized tools or platforms.

## Key Features
- Image-based ad classification by type (lifestyle, product shot, promo offer, etc.)
- Intelligent filtering of images based on content criteria
- AI-powered tagline generation for ad campaigns
- Sentiment analysis and feedback aggregation from customer reviews
- SQL-native workflow for unstructured data analysis

## Business Value

1. Enhanced Ad Campaign Optimization
2. Data-Driven Marketing Decisions
3. Operational Efficiency
4. Improved Campaign Performance

## Technical Components
The solution leverages several Snowflake Cortex AI-SQL functions:

- **FILE Data Type**: Process image files directly in Snowflake
- **AI_COMPLETE**: Generate creative taglines based on image content
- **AI_FILTER**: Identify images containing specific elements (e.g., coffee makers)
- **AI_CLASSIFY**: Categorize ads into predefined types
- **AI_AGG**: Aggregate and analyze customer feedback across ad categories





In [ ]:
# Add packages: matplotlib, snowflake-snowpark-python
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.types import StructType, StructField, StringType, IntegerType  
session=get_active_session()

session.query_tag = {"origin":"sf_sit-is", "name":"AISQL-HOL", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":3, "source":"notebook"}}
print(session)



In [ ]:
-- Set the correct schema context for accessing data
-- Data assets are in the default schema, not the notebooks schema
USE SCHEMA DEFAULT_SCHEMA;

A Snowflake stage containing all the images have been pre-loaded as part of the setup. A table named images containing the image_name,file url and other columns is created by extracting data from a Snowflake directory stage called ad_analytics.
Let us have a quick review of the dataset.


In [ ]:
SELECT
image_name, 
    img_file,
    file_url,
    last_modified
FROM images;

In [ ]:
DESCRIBE TABLE images;

## Creating taglines using AI_COMPLETE
Using the **Complete** function, let's create some fresh taglines for our new campaign's advertisement images.

In [ ]:
SELECT
    img_file,
    ai_complete('pixtral-large', prompt('Extract the text from this image and create an advertising tagline. Just return the tagline itself: {0}', img_file)),
    file_url,
    last_modified
FROM images;


## Filtering Ad images with  **AI_FILTER**
Past data has suggested that including coffee makers in ads boosts click-through rate. Let's use **AI_FILTER** to make sure we have ads with coffee makers in our ad campaign.

In [ ]:
SELECT img_file, relative_path, file_url, last_modified FROM images
WHERE AI_FILTER(prompt('does this image {0} have an espresso machine and/or a coffee pot/french press?', img_file))

## Classifying Ad Types with **AI_CLASSIFY**
Now let's prompt **AI_CLASSIFY** with a list of advertisement type categories so it can help classify the ads for us!

In [ ]:
CREATE OR REPLACE TABLE IMAGES_CLASSIFIED AS
SELECT
    img_file,
    image_name as file_name,
    --    relative_path as file_name,

     AI_CLASSIFY(
        prompt('please help me classify the type of advertisement within this coffee image {0}', img_file),
        ['long text', 'product shot', 'lifestyle', 'promo offer']
    ):labels[0]::text AS classification,
    
    file_url, 
    last_modified
FROM images;

SELECT * FROM IMAGES_CLASSIFIED;

In [ ]:
CREATE OR REPLACE TABLE IMAGES_CLASSIFIED AS
SELECT
    img_file,
   IMAGE_NAME as file_name,

     AI_CLASSIFY(
        prompt('please help me classify the type of advertisement within this coffee image {0}', img_file),
        ['long text', 'product shot', 'lifestyle', 'promo offer']
    ):labels[0]::text AS classification,
     AI_CLASSIFY(
        prompt('What is the advertising style depicted in this coffee-related image: {0}?', img_file),
        ['lifestyle', 'product_shot', 'promotional_offer', 'user_generated_content', 'artistic', 'informational']
    ):labels[0]::text AS ad_style,

    -- Emotional appeal
    AI_CLASSIFY(
        prompt('What emotional appeal does this coffee advertisement primarily use: {0}?', img_file),
        ['comfort_warmth', 'energy_excitement', 'luxury_prestige', 'convenience_efficiency', 'social_connection']
    ):labels[0]::text AS emotional_appeal,

    -- Target demographic
    AI_CLASSIFY(
        prompt('Based on this image, what is the likely target demographic for the coffee advertisement: {0}?', img_file),
        ['young_professionals', 'families', 'seniors', 'students', 'affluent_consumers']
    ):labels[0]::text AS target_demographic,
    file_url, 
    last_modified
FROM images;

SELECT * FROM IMAGES_CLASSIFIED;


### Technical Quality Assessment

Automate the assessment of image quality and compliance for digital asset review, content moderation, or advertising approval workflows using AISQL functions.

In [ ]:
CREATE OR REPLACE TABLE IMAGE_QUALITY_ASSESSMENT AS
SELECT
    img_file,
    image_name as file_name,
AI_CLASSIFY(
        prompt('How would you rate the technical image quality (focus, lighting, clarity, resolution) of this coffee image: {0}?', img_file),
        ['excellent_quality', 'good_quality', 'average_quality', 'poor_quality']
    ):labels[0]::text AS technical_quality,

    -- Brand safety
    AI_FILTER(
        prompt('Is this image appropriate for use in advertising and considered brand-safe? {0}', img_file)
    ) AS brand_safe,

    -- Professional photography standards
    AI_FILTER(
        prompt('Does this image meet professional photography standards (lighting, framing, clarity)? {0}', img_file)
    ) AS meets_standards,
file_url, 
    last_modified
FROM images;



In [ ]:
import streamlit as st
import altair as alt
import pandas as pd

# Fetch data from Snowflake (assuming session is already connected)
df = session.table("IMAGE_QUALITY_ASSESSMENT").to_pandas()
df.columns = [col.lower() for col in df.columns]
# Optional: ensure types are correct
df["technical_quality"] = df["technical_quality"].astype("str")
df["brand_safe"] = df["brand_safe"].astype("str")
df["meets_standards"] = df["meets_standards"].astype("str")

st.subheader("Distribution of Technical Image Quality")
quality_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("technical_quality:N", title="Technical Quality"),
    y=alt.Y("count():Q", title="Number of Images"),
    color="technical_quality:N",
    tooltip=["technical_quality", "count()"]
).properties(width=400)

st.altair_chart(quality_chart, use_container_width=True)

st.subheader("Brand Safety Assessment")
brand_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("brand_safe:N", title="Brand Safe"),
    y=alt.Y("count():Q", title="Number of Images"),
    color="brand_safe:N",
    tooltip=["brand_safe", "count()"]
).properties(width=400)

st.altair_chart(brand_chart, use_container_width=True)

st.subheader("Meets Professional Standards")
standards_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("meets_standards:N", title="Meets Standards"),
    y=alt.Y("count():Q", title="Number of Images"),
    color="meets_standards:N",
    tooltip=["meets_standards", "count()"]
).properties(width=400)

st.altair_chart(standards_chart, use_container_width=True)

# Optional: display table preview
st.subheader("Image Quality Assessment Results")
st.dataframe(df)


## Aggregating Reviews
Using functions like **AI_AGG()** to distill feedback, we can effectively analyze our 'Fuel Your Rise' ad campaign. This deeper understanding will enable us to craft more targeted advertising and enhance customer reach.

In [ ]:
csv_file_path = "@ad_analytics/IMPRESSIONS_WITH_REVIEWS.csv"



df= session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv(csv_file_path)
df.write.mode("overwrite").save_as_table("IMPRESSIONS_WITH_REVIEWS")

In [ ]:
SELECT * FROM IMPRESSIONS_WITH_REVIEWS;

In [ ]:
df=session.sql("""SELECT
image_type,

AI_AGG('Image Type: '|| image_type || '\n\nFeedback: ' || impression_review, 'score the sentiment of the feedback on a scale of 1 to 5 where 1 is very negative and 5 is very positive. Just write 1, 2, 3, 4, or 5. Return the average value only') as SENTIMENT,

AI_AGG('Image Type: '|| image_type || '\n\nFeedback: ' || impression_review, 'Analyze the feedback from these reviews and summarize the feedback in one sentence') as summary 
    
FROM IMPRESSIONS_WITH_REVIEWS
GROUP BY image_type""").to_pandas()

st.subheader("AI Sentiment Summary by Image Type")
st.dataframe(df)
# Convert sentiment column to numeric (if AI_AGG returns it as string)
df['sentiment'] = pd.to_numeric(df['SENTIMENT'], errors='coerce')

# Plot chart
st.bar_chart(df.set_index('IMAGE_TYPE')['sentiment'])

# Show summary text (optional)
st.subheader("Summaries")
for _, row in df.iterrows():
    st.markdown(f"**{row['IMAGE_TYPE']}**: {row['SUMMARY']}")

## Performance Analytics Integration

In [ ]:
adcsv_file_path = "@ad_analytics/AD_PERFORMANCE.csv"


df_AD_PERFORMANCE= session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv(adcsv_file_path)

df_AD_PERFORMANCE.write.mode("overwrite").save_as_table("AD_PERFORMANCE")

In [ ]:
SELECT * FROM AD_PERFORMANCE;

## Performance-Image Correlation

In [ ]:
SELECT 
TO_FILE(ia.FILE_URL) Img_File,
    ad_style,
    emotional_appeal,
    AVG(ctr_percent) as avg_ctr,
    AVG(conversion_rate_percent) as avg_conversion,
    AVG(roas) as avg_roas,
      AI_AGG(
        'Ad Style: ' || ad_style || 
        '\nEmotional Appeal: ' || emotional_appeal || 
        '\nCTR: ' || ctr_percent || 
        '\nConversion Rate: ' || conversion_rate_percent || 
        '\nROAS: ' || roas,
        'Analyze the relationship between ad style, emotional appeal, and performance metrics. Summarize key performance patterns in 1 line.'
    ) AS insights
   FROM IMAGES_CLASSIFIED ia
JOIN ad_performance ap ON ia.file_name = ap.IMAGE
GROUP BY ad_style, emotional_appeal,ia.FILE_URL;

### OPTIMIZATION RECOMMENDATIONS FROM AD PERFORMANCE

In [ ]:
create or replace temporary view Optimization_recommendation_vw  as SELECT
    TO_FILE(ic.file_url) Img_File,
    ic.ad_style,
    ic.emotional_appeal,
    ap.ctr_percent,
    ap.conversion_rate_percent,
    ap.roas,

    -- Optimization recommendation
    AI_CLASSIFY(
      'CTR: ' || ap.ctr_percent || 
      ', Conversion Rate: ' || ap.conversion_rate_percent || 
      ', ROAS: ' || ap.roas, 
      ['increase_budget', 'add_creativity', 'change_targeting', 'pause_ad']
    ) AS recommended_action,

    -- Creative improvement suggestions
    AI_COMPLETE(
      'pixtral-large',
      prompt('The ad image {0} is underperforming. Suggest 3 specific creative changes to improve it.', ic.img_file)
    ) AS creative_improvements

FROM images_classified ic
JOIN ad_performance ap
    ON ic.file_name = ap.IMAGE
WHERE ap.ctr_percent < 20.0 OR ap.roas < 10.0;


In [ ]:
import streamlit as st
import pandas as pd

df=session.sql("select * from Optimization_recommendation_vw").to_pandas()
df.columns = [col.lower() for col in df.columns]

st.dataframe(df[['img_file', 'ad_style', 'creative_improvements']])
df.head()

scatter_chart = alt.Chart(df).mark_circle(size=80).encode(
    x=alt.X('ctr_percent:Q', title='CTR (%)'),
    y=alt.Y('roas:Q', title='ROAS'),
    color='recommended_action:N',
    tooltip=['ad_style', 'ctr_percent', 'conversion_rate_percent', 'roas', 'recommended_action']
).properties(
    title="CTR vs ROAS Colored by Recommended Action"
).interactive()

st.altair_chart(scatter_chart, use_container_width=True)



## In Summary


Snowflake (AI SQL unlocks unstructured Data Analysis, all with the familiarity of SQL This solution demonstrated how to leverage Snowflake's AISQL capabilities to analyze ad campaign images and impression data.


## Business Value achieved

1. Enhanced Ad Campaign Optimization
2. Data-Driven Marketing Decisions
3. Operational Efficiency
4. Improved Campaign Performance